# Fit the sinusoidal amplitudes using the RooComponentsPDF class

Here we use the RooComponentsPDF class to provide fast Simulated normalisation integration.
First load the fit classes.

In [ ]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
#%jsroot

Use an instance of the FitManager class to organise the fit and give it an ouput directory.

In [ ]:
fm = ROOT.FitManager()
fm.SetUp().SetOutDir("outObs2/");

Now load the experimentally measured variables (or observables). I must use the name of the variables in the tree.

Note the slighlty different notation for the Category PolState. If any events had a PolState of 0 for example, then they would not be included in the fit.

In [ ]:
fm.SetUp().LoadVariable("Phi[-180,180]"); 
fm.SetUp().LoadVariable("Pol[0,1]"); 
fm.SetUp().LoadCategory("PolState[Polp=1,Pol0=0,Polm=-1]");

If I want to use any other variable, or example to apply a cut, I must load it as an AuxVar. The limits given will be applied to the datset. Here we will not apply it, but it is left commented out to show what you could do.

In [ ]:
#fm.SetUp().LoadAuxVar("M1[0,10]"); //Load Aux Var, limits used as cut 
#fm.SetUp().AddCut("M1>2"); //Additional cut based on vars or aux vars

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights. This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards. The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
fm.SetUp().SetIDBranchName("fgID");

Create and load into the fit manager my PDF class. Phi, Pol and PolState all match the variables loaded above.

Here we define the functional parts seperately although they could be done on the fly on the FactoryPDF

In [ ]:
fm.SetUp().LoadFormula("SIN2=@PolState[]*@Pol[]*sin(2*(@Phi[])/57.29578)");
fm.SetUp().LoadFormula("COS2=@PolState[]*@Pol[]*cos(2*(@Phi[])/57.29578)");

Here we load the fit parameters on the fly in the factory PDF e.g. A[0,-1,1] initial value 0 and range between -1 and 1. If A appears multiple times I should only include the [0,-1,1] in the first instance and then just A from then on.

WE could have Loaded them explicitly first. This may help reading in more complicate functions. e.g.

    fm.SetUp().LoadParameter("A[0,-1,1]");
    
Then just use A rather than A[0,-1,1] in FactoryPDF


    1 + A*COS2 + B*SIN2 (first argument = the 1 )
    
Note : seperates different products = +
     ; seperates different terms in the product = *
     
Note the 1 in "SigAsym(1" corresponds to the 1 + in the formula. This may also be set to 0 if the formula does not have a 1+ at the start.

In [ ]:
fm.SetUp().FactoryPDF("RooComponentsPDF::SigAsym(1,{Phi,Pol,PolState},=A[0,-1,1];COS2:B[0,-1,1];SIN2)");
fm.SetUp().LoadSpeciesPDF("SigAsym",1);
fm.SetUp().WS().Print("v")

Split the data into 4 Eg bins and perform 4 fits (in parallel with PROOF). At the end we will create Graphs of the fit parameters as a function of Eg.

In [ ]:
 fm.Bins().LoadBinVar("Eg",4,3,4);

Now I load my data, here I will use both data to fit and MC data for the integral calculation. Note in the latter case the string "SigAsym" matches the name given to the PDF object above. In principal I could fit multiple PDFs and give them each different MC data.

The strings are treename, filename and PDF name (simulation only).

In [ ]:
fm.LoadData("MyModel","Data.root");
fm.LoadSimulated("MyModel","MC.root","SigAsym");

Now I attach the weights from my sPlot fit. I want to use the signal weights which were given the name "Signal" in the sPlot notebook. 

Also there are two possible sets of weights from using a Gaussian or simulated PDF. You can switch between either here.

In [ ]:
fm.Data().LoadWeights("Signal","outSplot/Tweights.root");
#fm.Data().LoadWeights("Signal","outSplotSim/Tweights.root"); #use weights from simulated PDF

I can provide additional options to speed up the fit. Here I use 3 CPUs which parallelises the likelihood calculation.

And I set optimise to 1 which calculates and caches formulas in the first interation, then uses the cached values in future iterations of the likelihood calculation. This is well matched to the ComponentsPDF .

In [ ]:
fm.SetUp().AddFitOption(ROOT.RooFit.NumCPU(3));
fm.SetUp().AddFitOption(ROOT.RooFit.Optimize(1));

Now run the fits. I use the Process classes which allow me to choose between running directly here on a single core or multicore via PROOF-lite. It doesn't make sense to run with PROOF unless multiple splits have been defined with LoadBinVar or you are using Bootstrap, in which case you should relate the number of cores requested to the number of splits or bootstraps.

** For me the programme hangs at this cell once the fit is done and the plots created unless I use SetRedirectOutput which sends the text to a log file in the OutDir

In [ ]:
ROOT.Proof.Go(fm,4)

All plots and fit parameters will be saved into the output directory Results.root file. If LoadBinVar splits were applied then they will be in directories related to the bin name.

If the plot does not appear you can try removing the %jsroot in the first cell. Unfortunately this will mean the histograms will not be interactive.

### Graphing the fit results

Here we use the GraphParameters macro to make TGraphErrors of the parameter fit results. We need to supply the directory name and the binned variable we want to plot against.

In [ ]:
ROOT.gROOT.ProcessLine(".L GraphParameters.C+")
ROOT.GraphParameters(fm.SetUp().GetOutDir(),"Eg")

In [ ]:
graphFile=ROOT.TFile.Open(fm.SetUp().GetOutDir().Data()+"ParGraphsEg.root")
canvas=ROOT.TCanvas()
canvas.Divide(1,3)
canvas.cd(1)
graphFile.Get("A").Draw("ape")
canvas.cd(2)
graphFile.Get("B").Draw("ape")
canvas.cd(3)
graphFile.Get("Yld_SigAsym").Draw("ape")
canvas.Draw()